In [2]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
# 라이브러리 가져오기
import requests
import pandas as pd
import os
import dotenv

In [4]:
dotenv.load_dotenv()
my_key = os.getenv('KAKAO_RESTFULL_KEY')

In [5]:
# Kakao Map API 통해 위도, 경도 데이터 가져오는 함수를 정의
def get_geocoding(place):
    try:
        url = f"https://dapi.kakao.com/v2/local/search/address"
        headers = {"Authorization": f"KakaoAK {my_key}"}
        params = {
            'query' : place
        }
        response = requests.get(url, headers=headers,params=params)
        result = response.json()
        if result["documents"]:
            return result["documents"][0]["y"], result["documents"][0]["x"]
        else:
            return None,None
    except:
        return None,None

In [6]:
lat = []  #위도
lng = []  #경도

# 장소(주소) 리스트
places = ["서울특별시 종로구 세종대로 175", 
          "서울특별시 서초구 서초동 700", 
          "부산광역시 해운대구 해운대해변로 264"]

i=0
for place in places:   
    i = i + 1
    try:
        print(i, place)
        # get_geocoding 함수의 리턴값 호출하여 geo_location 변수에 저장
        place_lat, place_lon = get_geocoding(place)
        lat.append(place_lat)
        lng.append(place_lon)
        
    except:
        lat.append('')
        lng.append('')

# 데이터프레임으로 변환하기
df = pd.DataFrame({'위도':lat, '경도':lng}, index=places)

df

1 서울특별시 종로구 세종대로 175
2 서울특별시 서초구 서초동 700
2 서울특별시 서초구 서초동 700
3 부산광역시 해운대구 해운대해변로 264
3 부산광역시 해운대구 해운대해변로 264


,위도,경도
서울특별시 종로구 세종대로 175,37.5718478584908,126.976168275947
서울특별시 서초구 서초동 700,37.4810862955299,127.015245160054
부산광역시 해운대구 해운대해변로 264,35.1591069824231,129.160283786856


In [7]:
df = pd.read_csv('https://raw.githubusercontent.com/pia222sk20/python/main/data/sample.csv'
                 ,encoding='cp949')
df_simple = df.loc[:,['상호명','도로명주소']]
df_simple.head()

,상호명,도로명주소
0,하나산부인과,경기도 안산시 단원구 달미로 10
1,타워광명내과의원,서울특별시 강남구 언주로30길 39
2,조정현신경외과의원,경기도 시흥시 중심상가로 178
3,한귀원정신과의원,부산광역시 수영구 수영로 688
4,더블유스토어수지점,경기도 용인시 수지구 문정로 32


In [8]:
#위도 경고를 추가  apply
# df_simple['위경도'] = df_simple['도로명주소'].apply(lambda x : get_geocoding(x))
df_simple['도로명주소'].index


RangeIndex(start=0, stop=91335, step=1)

In [9]:
import random
random_index = random.sample(range(91335),50)  # radom_index -> random_index로 수정
random_index[:10]

[58785, 365, 14337, 8134, 56013, 51342, 70632, 90288, 8464, 86621]

In [10]:
df_simple = df_simple.dropna()

In [11]:
df_simple_50 =  df_simple.iloc[random_index]
results = [get_geocoding(addr) for addr in df_simple_50['도로명주소'].values]
results[:10]

[('37.4109641947715', '127.129259400691'),
 ('35.496200049285', '128.748385254084'),
 ('37.5123997269638', '126.703594090796'),
 ('37.7154845077667', '128.830649970312'),
 ('37.2921573223499', '127.050652669552'),
 ('37.3956881776014', '126.645446887038'),
 ('35.1593255869405', '129.144241981158'),
 ('37.4004659332759', '127.106812648944'),
 ('37.4619715724957', '126.879970546974'),
 ('35.8139175973326', '128.524690381173')]

In [21]:
df_simple_50.loc[:,'위도'], df_simple_50.loc[:,'경도'] = zip(*results)

In [12]:
# 랜덤으로 선택된 50개의 데이터 추출
df_simple_50 = df_simple.iloc[random_index]

# 위도, 경도 데이터 가져오기
results = []
for addr in df_simple_50['도로명주소'].values:
    if pd.notna(addr):  # 주소가 None이 아닌 경우만 처리
        result = get_geocoding(addr)
        results.append(result)
    else:
        results.append((None, None))

# 위도, 경도 데이터를 DataFrame에 추가
lat_lng = pd.DataFrame(results, columns=['위도', '경도'])
df_simple_50 = df_simple_50.reset_index(drop=True)  # 인덱스 초기화
df_simple_50['위도'] = lat_lng['위도']
df_simple_50['경도'] = lat_lng['경도']

# 결과 확인
print("데이터 샘플 (처음 5개):")
print(df_simple_50[['상호명', '도로명주소', '위도', '경도']].head())

데이터 샘플 (처음 5개):
         상호명                  도로명주소                위도                경도
0  분당아이조아한의원   경기도 성남시 분당구 성남대로 912  37.4109641947715  127.129259400691
1       제일의원       경상남도 밀양시 노상하4길 4   35.496200049285  128.748385254084
2     애나동물병원      인천광역시 부평구 마장로 383  37.5123997269638  126.703594090796
3       유명약국     강원도 강릉시 성산면 구산길 59  37.7154845077667  128.830649970312
4     광교연세안과  경기도 수원시 영통구 센트럴타운로 98  37.2921573223499  127.050652669552


In [13]:
df_simple_1000= df_simple.iloc[random_index]
results = [get_geocoding(addr) for addr in df_simple_50['도로명주소'].values]
results[:10]

[('37.4109641947715', '127.129259400691'),
 ('35.496200049285', '128.748385254084'),
 ('37.5123997269638', '126.703594090796'),
 ('37.7154845077667', '128.830649970312'),
 ('37.2921573223499', '127.050652669552'),
 ('37.3956881776014', '126.645446887038'),
 ('35.1593255869405', '129.144241981158'),
 ('37.4004659332759', '127.106812648944'),
 ('37.4619715724957', '126.879970546974'),
 ('35.8139175973326', '128.524690381173')]

In [14]:
df_simple_50.head()

,상호명,도로명주소,위도,경도
0,분당아이조아한의원,경기도 성남시 분당구 성남대로 912,37.4109641947715,127.129259400691
1,제일의원,경상남도 밀양시 노상하4길 4,35.496200049285,128.748385254084
2,애나동물병원,인천광역시 부평구 마장로 383,37.5123997269638,126.703594090796
3,유명약국,강원도 강릉시 성산면 구산길 59,37.7154845077667,128.830649970312
4,광교연세안과,경기도 수원시 영통구 센트럴타운로 98,37.2921573223499,127.050652669552


In [15]:
%pip install folium

Note: you may need to restart the kernel to use updated packages.


In [16]:
# 모든 컬럼 출력 확인
for i in df_simple_50:
    print(f"컬럼명: {i}")

print("\n처음 5개 데이터의 모든 정보:")
print(df_simple_50.head())

컬럼명: 상호명
컬럼명: 도로명주소
컬럼명: 위도
컬럼명: 경도

처음 5개 데이터의 모든 정보:
         상호명                  도로명주소                위도                경도
0  분당아이조아한의원   경기도 성남시 분당구 성남대로 912  37.4109641947715  127.129259400691
1       제일의원       경상남도 밀양시 노상하4길 4   35.496200049285  128.748385254084
2     애나동물병원      인천광역시 부평구 마장로 383  37.5123997269638  126.703594090796
3       유명약국     강원도 강릉시 성산면 구산길 59  37.7154845077667  128.830649970312
4     광교연세안과  경기도 수원시 영통구 센트럴타운로 98  37.2921573223499  127.050652669552


In [17]:
for idx, row in df_simple_50.iterrows():
    print(idx, row['위도'], row['경도'])
    break

0 37.4109641947715 127.129259400691


In [18]:
import folium

m = folium.Map(location=(35.8425738690814, 127.140054344904))  # 최초지도 로드할때. 중심좌표
for idx, row in df_simple_50.iterrows():
    print(idx, row['위도'], row['경도'])
folium.Marker(
    location=[35.8425738690814, 127.140054344904],
    popup='고려마취통증의학과의원',
    tooltip='병원'
).add_to(m)

folium.Marker(
    location=[37.8687445127498,	127.714867915848],
    popup='백세약국',
    tooltip='약국'
).add_to(m)


m.save('map.html')


0 37.4109641947715 127.129259400691
1 35.496200049285 128.748385254084
2 37.5123997269638 126.703594090796
3 37.7154845077667 128.830649970312
4 37.2921573223499 127.050652669552
5 37.3956881776014 126.645446887038
6 35.1593255869405 129.144241981158
7 37.4004659332759 127.106812648944
8 37.4619715724957 126.879970546974
9 35.8139175973326 128.524690381173
10 37.4968946341351 127.14994730411
11 37.4988301355807 127.059558169501
12 37.3082616940569 126.85202762807
13 37.5067411635893 127.005520174535
14 34.999290659807 126.709947054752
15 35.5435402133644 129.309543930585
16 37.4151399550011 126.677813880127
17 36.8617475651202 127.444835310721
18 37.37056133067 127.107511843851
19 35.5470632424327 129.309686897918
20 37.4716932381309 126.631818357765
21 37.5668276642327 126.963975289329
22 35.9290959010644 126.959442660367
23 35.8411382714422 127.131588969832
24 36.3212144016661 127.393531434484
25 35.1287004284271 128.972490130663
26 37.3070818396581 126.850134449613
27 37.49606585853

In [19]:
for i in df_simple_50:
    print(1)

1
1
1
1


In [22]:
import folium

# 서울 중심 좌표로 지도 생성
m = folium.Map(location=[37.5665, 126.9780], zoom_start=7)

# df_simple_50의 각 위치에 마커 추가
for idx, row in df_simple_50.iterrows():
    if pd.notna(row['위도']) and pd.notna(row['경도']):  # None 값 체크
        folium.Marker(
            location=[float(row['위도']), float(row['경도'])],
            popup=row['상호명'],
            tooltip=row['도로명주소']
        ).add_to(m)

# 지도 저장
m.save('map.html')

# 지도 파일 열기
import webbrowser
webbrowser.open('map.html')

True